In [ ]:
# Install the necessary library to interact with BigQuery
%pip install pandas-gbq
%pip install --upgrade google-genai

In [ ]:
import pandas_gbq
import bigframes.pandas as bf
import os

In [ ]:
# Google Cloud Project
project_id = "adcz-adoki-poc"
location_id = "europe-west1"

from google.cloud import bigquery
client = bigquery.Client(project=project_id, location=location_id)

### Get data from BigQuery
- using local/Workbanch computing

In [ ]:
# Use local computing power
sql = "SELECT property_id, address_latitude, address_longitude FROM subs_property_data.property WHERE address_latitude > 0"
df_property = bf.read_gbq(sql).head(10)

display(df_property)

### Historické počasí (včerejšek) pro GPS souřadnice
Získáme denní metriky (max/min teplota, srážky) z Open-Meteo Archive API pro datum včera (časová zóna Europe/Prague).

In [ ]:
import math
import time
import requests
import pprint
from datetime import datetime, timedelta, date


# Připravíme unikátní souřadnice (předpoklad: sloupce address_latitude, address_longitude existují v df_property)
coords_df = df_property[['address_latitude', 'address_longitude']].dropna().drop_duplicates().rename(columns={
    'address_latitude': 'lat',
    'address_longitude': 'lon'
})

# Definice hodinových parametrů podle dokumentace
hourly_params = [
    'temperature_2m',          # °C
    'relative_humidity_2m',    # %
    'rain',                    # mm
    'snowfall',                # cm new snow (we will also try snow_depth if needed)
    'snow_depth',              # snow depth (documentation: some APIs provide)
    'windspeed_10m'            # km/h
]

hourly_results = []
base_url_hourly = "https://archive-api.open-meteo.com/v1/archive"
today = datetime.now()
start_date = (today - timedelta(days=2)).date()
end_date = start_date

first_coord_debug_done = False
for row in coords_df.itertuples(index=False):
    lat, lon = row.lat, row.lon
    params = {
        'latitude': lat,
        'longitude': lon,
        'start_date': start_date,
        'end_date': end_date,
        'hourly': ','.join(hourly_params),
        'timezone': 'Europe/Prague'
    }
    try:
        r = requests.get(base_url_hourly, params=params, timeout=60)
        r.raise_for_status()
        data = r.json()
        hourly = data.get('hourly')
        if not hourly:
            print(f"⚠️ Chybí 'hourly' blok pro {lat},{lon}")
            continue
        times = hourly.get('time', [])

        # Debug pro první koordinát – zjistíme délky jednotlivých polí
        if not first_coord_debug_done:
            print("--- DEBUG první koordináty ---")
            print("Počet timestamps:", len(times))
            for p in hourly_params:
                arr = hourly.get(p)
                print(f"{p}: length={len(arr) if isinstance(arr, list) else 'None'}")
            print("Ukázka prvních 5 timestamps:", times[:5])
            first_coord_debug_done = True

        # Pokud některé pole má kratší délku než times, doplníme None a upozorníme
        max_len = len(times)
        normalized = {}
        for p in hourly_params:
            arr = hourly.get(p, []) or []
            if len(arr) != max_len:
                print(f"⚠️ Varování: {p} má délku {len(arr)} != {max_len} (doplním None)")
                arr = (arr + [None] * max_len)[:max_len]
            normalized[p] = arr

        for i, ts in enumerate(times):
            # Rozdělení na datum a hodinu (formát 'YYYY-MM-DDTHH:MM')
            try:
                date_part, time_part = ts.split('T')
                hour_part = time_part.split(':')[0]
            except ValueError:
                continue
            record = {
                'lat': lat,
                'lon': lon,
                'date': date_part,
                'hour': int(hour_part),
                'temperature': normalized['temperature_2m'][i],
                'relative_humidity': normalized['relative_humidity_2m'][i],
                'rain': normalized['rain'][i],
                'snow_depth': None,  # nastavíme níže
                'wind_speed': normalized['windspeed_10m'][i],
            }
            # Sněh: preferuj snow_depth, fallback snowfall
            if normalized['snow_depth'][i] is not None:
                record['snow_depth'] = normalized['snow_depth'][i]
            elif normalized['snowfall'][i] is not None:
                record['snow_depth'] = normalized['snowfall'][i]

            hourly_results.append(record)
        # Krátká pauza mezi koordinátami (ne uvnitř hodin) pro šetrnost
        time.sleep(0.15)

    except Exception as e:  # noqa: BLE001
        print(f"Chyba (hourly) pro {lat},{lon}: {e}")

hourly_weather_df = pd.DataFrame(hourly_results)

# Převod df_property na pandas + mapování lat/lon -> property_id (může být mnoho stejných souřadnic)
prop_coord_map = df_property.to_pandas()[['property_id', 'address_latitude', 'address_longitude']].dropna()

# Merge: mnohonásobné přiřazení property_id pro shodné souřadnice
hourly_merged = hourly_weather_df.merge(
    prop_coord_map,
    left_on=['lat', 'lon'],
    right_on=['address_latitude', 'address_longitude'],
    how='left'
)

# Výběr požadovaných sloupců a přejmenování
final_hourly_df = hourly_merged[['property_id', 'date', 'hour', 'temperature', 'relative_humidity', 'rain', 'snow_depth', 'wind_speed']].copy()

# Typové úpravy
if not final_hourly_df.empty:
    final_hourly_df['hour'] = final_hourly_df['hour'].astype(int)

print(f"Hodinových záznamů počasí celkem: {len(final_hourly_df)}")

display(final_hourly_df.head(30))

### Import do BigQuery
Target tabulka: 'property_weather'
- if does not exist then create one

In [ ]:
from google.cloud import bigquery
from google.api_core.exceptions import NotFound
import pandas as pd
from datetime import datetime

# --- Konfigurace ---
project_id = "adcz-adoki-poc"  # pokud chceš dynamicky, můžeš načíst z env proměnné
dataset_id = "demo_real_estate"
table_id = "property_weather"
full_table_id = f"{project_id}.{dataset_id}.{table_id}"

# Ověření, že máme zdrojový DataFrame
if 'final_hourly_df' not in globals():
    raise RuntimeError("final_hourly_df neexistuje. Spusť nejprve sekci s hodinovými daty.")

# Kopie a úprava dat
bq_df = final_hourly_df.copy()

# Přidáme ingestion timestamp
bq_df['ingested_at'] = datetime.utcnow()

# Normalizace typů a NaN -> None
for col in ['temperature','relative_humidity','rain','snow_depth','wind_speed']:
    if col in bq_df.columns:
        bq_df[col] = pd.to_numeric(bq_df[col], errors='coerce')

# property_id jako string (pro konzistenci a aby se neztratily případné jiné formáty)
if 'property_id' in bq_df.columns:
    bq_df['property_id'] = bq_df['property_id'].astype(str)

# hour integer
if 'hour' in bq_df.columns:
    bq_df['hour'] = bq_df['hour'].astype(int)

# date na date objekt (pokud je string)
if 'date' in bq_df.columns:
    try:
        bq_df['date'] = pd.to_datetime(bq_df['date']).dt.date
    except Exception as e:  # noqa: BLE001
        print("Varování: nepodařilo se konvertovat 'date' na date:", e)

bq_df = bq_df.where(pd.notnull(bq_df), None)

client = bigquery.Client(project=project_id)

# --- Dataset ---
try:
    client.get_dataset(f"{project_id}.{dataset_id}")
    print(f"Dataset '{dataset_id}' OK")
except NotFound:
    dataset = bigquery.Dataset(f"{project_id}.{dataset_id}")
    dataset.location = "EU"  # uprav dle potřeby
    dataset = client.create_dataset(dataset)
    print(f"Vytvořen dataset: {dataset.full_dataset_id}")

# --- Tabulka ---
try:
    client.get_table(full_table_id)
    table_exists = True
    print(f"Tabulka '{full_table_id}' existuje – data budou APPEND.")
except NotFound:
    table_exists = False
    schema = [
        bigquery.SchemaField("property_id", "STRING"),
        bigquery.SchemaField("date", "DATE"),
        bigquery.SchemaField("hour", "INT64"),
        bigquery.SchemaField("temperature", "FLOAT64"),
        bigquery.SchemaField("relative_humidity", "FLOAT64"),
        bigquery.SchemaField("rain", "FLOAT64"),
        bigquery.SchemaField("snow_depth", "FLOAT64"),
        bigquery.SchemaField("wind_speed", "FLOAT64"),
        bigquery.SchemaField("ingested_at", "TIMESTAMP"),
    ]
    table = bigquery.Table(full_table_id, schema=schema)
    table = client.create_table(table)
    print(f"Vytvořena tabulka: {table.full_table_id}")

# --- Load Job ---
job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_APPEND)

load_job = client.load_table_from_dataframe(bq_df, full_table_id, job_config=job_config)
print("Nahrávám data do BigQuery...")
load_job.result()  # čekáme na dokončení

# Kontrola počtu řádků
table = client.get_table(full_table_id)
print(f"Hotovo. Tabulka nyní obsahuje {table.num_rows} řádků.")

# Zobrazíme malý výřez
display(bq_df.head())